## Hands-on
### Quantifying and Reducing Gender Stereotypes in Portuguese Word Embeddings

Based in the following paper: Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings by Tolga Bolukbasi, Kai-Wei Chang, James Zou, Venkatesh Saligrama, and Adam Kalai. Proceedings of NIPS 2016.

In [1]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions_pt

### Part1 
#### Gender Bias in Word Embedding


- #### Step 1: Load data

In [2]:
# load google news word2vec
E = WordEmbedding('./embeddings/cbow_s300.txt')


*** Reading data from ./embeddings/cbow_s300.txt
(934966, 300)
934966 words of dimension 300 : </s>, ,, de, ., ..., indispô-la, microquimerismo, matro, sentar-se-ia
934966 words of dimension 300 : </s>, ,, de, ., ..., indispô-la, microquimerismo, matro, sentar-se-ia


In [3]:
# load professions
professions = load_professions_pt()
profession_words = [p[0] for p in professions]

Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


- #### Step 2: Define Gender Direction

In [4]:
# gender direction
v_gender = E.diff('ela', 'ele')

- #### Step 3: Generating analogies of "Homem: x :: Mulher : y"

In [5]:
"""a_gender = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender:
    print(a+"-"+b)"""

'a_gender = E.best_analogies_dist_thresh(v_gender)\n\nfor (a,b,c) in a_gender:\n    print(a+"-"+b)'

In [5]:
professions_pair=[]

with open('./data/professions_analogies_pt.txt', "r") as f:
    for line in f:
        line=line.replace("\n","").replace("(","").replace(")","").replace("'","").replace(" ","")
        male=line.split(",")[1]
        fema=line.split(",")[0]
        professions_pair.append([male,fema])

In [6]:
def evaluate_w2c(E,professions_pair):
       
   #def evaluete_tuples(E,professions_pair):
    tmp_file='./data/temp.bin'
    E.save_w2v(tmp_file)
    print ("saved")
    from gensim.models import KeyedVectors
    tmp_file='./data/temp.bin'
    word_vectors = KeyedVectors.load_word2vec_format(tmp_file, binary=True)
    with open("./data/professions_analogies_pt.txt", "r") as ins:
        for line in ins:
            line = line.replace("\n","").replace("(","").replace(")","").replace("'","").replace(" ","")
            male = line.split(",")[1]
            fema = line.split(",")[0]
            professions_pair.append([male,fema])

    len_= len(professions_pair)
    acc = 0
    for p in professions_pair:
        male=p[0]
        fema=p[1]
        #word_vectors.
        if (male in word_vectors.vocab and fema in word_vectors.vocab):
            x = word_vectors.most_similar(positive=['mulher', male], negative=['homem'])[0][0]
            print (male+"="+x+"-"+fema)
            if x== fema:
                acc+=1

    out = acc/float(len_)
    print (acc)
    print (len_)
    print ("accuracy =",out)
    return out



#evaluate_w2c(E,professions_pair)

- #### Step 4: Analyzing gender bias in word vectors associated with professions

In [7]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

male, female = sp[0:20], sp[-20:]
biased_pairs = []
for i, j in zip(male, female):
    biased_pairs.append([i[1],j[1]])
    
evaluate_w2c(E,biased_pairs)

saved
embaixador=esposa-embaixador
contador=esposa-contador
arqueólogo=enfermeira-arqueólogo
analista=enfermeira-analista
antropólogo=esposa-antropólogo
ajudante=enfermeira-ajudante
arcebispo=esposa-arcebispo
aventureiro=esposa-aventureiro
arquiteto=esposa-arquiteto
artista=enfermeira-artista
ator=atriz-ator
vereadora=enfermeira-vereadora
advogada=enfermeira-advogada
conhecida=conhecido-conhecida
assassina=cadela-assassina
administradora=enfermeira-administradora
atriz=socialite-atriz
advogado=advogada-advogada
arquiteto=esposa-arquiteta
ator=atriz-atriz
biologo=esposa-biologa
blogueiro=dama-de-companhia-blogueira
bombeiro=enfermeira-bombeira
cabeleireiro=cabeleireira-cabeleireira
camareiro=esposa-camareira
cantor=namorada-cantora
consultor=enfermeira-consultora
contador=esposa-contadora
coordenador=coordenadora-coordenadora
coreografo=esposa-coreografa
costureiro=esposa-costureira
cozinheiro=enfermeira-cozinheira
dançarino=dançarina-dançarina
decorador=esposa-decoradora
diagramador=ex

0.18181818181818182

### Part 2
#### Debias

- #### Step 5:

In [8]:
from debiaswe.debias import debias

In [9]:
with open('./data/definitional_pairs_pt.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs_pt.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed_pt.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['mulher', 'homem'], ['garota', 'garoto'], ['ela', 'ele'], ['mae', 'pai'], ['filha', 'filho'], ['moca', 'moco'], ['femea', 'macho'], ['dela', 'dele'], ['propria', 'proprio']]
gender specific 141 ['atriz', 'atrizes', 'tia', 'tias', 'bacharel', 'bacharela', 'bailarina', 'barbearia', 'baritono', 'barba']


In [10]:
E_new = debias(E, gender_specific_words, defs, equalize_pairs)

sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

male, female = sp_debiased[0:20], sp_debiased[-20:]
debiased_pairs = []
for i, j in zip(male, female):
    debiased_pairs.append([i[1],j[1]])



934966 words of dimension 300 : </s>, ,, de, ., ..., indispô-la, microquimerismo, matro, sentar-se-ia
{('FRATERNIDADE', 'SORORIDADE'), ('Proprio', 'Propria'), ('PATERNIDADE', 'MATERNIDADE'), ('irmaos', 'irmas'), ('paternidade', 'maternidade'), ('PAIS', 'MAES'), ('IRMAO GEMEO', 'IRMA GEMEA'), ('Aluno', 'Aluna'), ('VEREADOR', 'VEREADORA'), ('principe', 'princesa'), ('pai', 'mãe'), ('menino', 'menina'), ('EMPRESARIO', 'EMPRESARIA'), ('irmao gemeo', 'irma gemea'), ('testosterona', 'estrogenio'), ('FILHOS', 'FILHAS'), ('filhos', 'filhas'), ('Meninos', 'Meninas'), ('Homem', 'Mulher'), ('presidente', 'presidenta'), ('TIO', 'TIA'), ('REIS', 'RAINHAS'), ('PROPRIO', 'PROPRIA'), ('Cavalheiro', 'Dama'), ('Machos', 'Femeas'), ('vereador', 'vereadora'), ('PADRE', 'FREIRA'), ('Vereador', 'Vereadora'), ('monastério', 'convento'), ('MONASTÉRIO', 'CONVENTO'), ('MACHOS', 'FEMEAS'), ('Pai', 'Mae'), ('ELE', 'ELA'), ('Pai', 'Mãe'), ('tio', 'tia'), ('MENINOS', 'MENINAS'), ('MACHO', 'FEMEA'), ('Macho', 'Femea

In [13]:
evaluate_w2c(E_new,debiased_pairs)

saved
embaixador=empresaria-embaixador
analista=mae-analista
contador=irma-contador
advogada=mãe-advogada
ajudante=irma-ajudante
arqueólogo=empresaria-arqueólogo
antropólogo=empresaria-antropólogo
arcebispo=irma-arcebispo
aventureiro=dama-aventureiro
artista=empresaria-artista
administradora=mãe-administradora
arquiteto=empresaria-arquiteto
ator=empresaria-ator
assassina=mãe-assassina
conhecida=conhecido-conhecida
vereadora=tia-vereadora
atriz=irma-atriz
advogado=mae-advogada
arquiteto=empresaria-arquiteta
ator=empresaria-atriz
biologo=empresaria-biologa
blogueiro=empresaria-blogueira
bombeiro=mãe-bombeira
cabeleireiro=irma-cabeleireira
camareiro=dama-camareira
cantor=empresaria-cantora
consultor=empresaria-consultora
contador=irma-contadora
coordenador=presidenta-coordenadora
coreografo=empresaria-coreografa
costureiro=irma-costureira
cozinheiro=mãe-cozinheira
dançarino=irma-dançarina
decorador=irma-decoradora
diagramador=irma-diagramadora
diretor=empresaria-diretora
empresario=empres

0.030303030303030304